# Tarjeta de datos 5

Hipótesis 5: *¿Qué cantidad de placas solares fotovoltaicas se necesitarían instalar para abastecer el consumo eléctrico de los hogares de Castilla-La Mancha en el año 2028?*

Esta hipótesis es interesante para conocer cuál es el verdadero potencial y la rentabilidad de uno de los dispositivos de aprovechamiento de energía renovables más comunes y utilizados actualmente, las placas solares fotovoltaicas. 

El objetivo es obtener una tarjeta de datos con la siguente estructura:

| Provincia   | Producción eléctrica anual (MWh) | Consumo (MWh) Año X | Consumo (MWh) Año X+1 | ... |
|-------------|----------------------------------|---------------------|-----------------------|-----|
| Albacete    |                                  |                     |                       |     |
| Ciudad Real |                                  |                     |                       |     |
| Cuenca      |                                  |                     |                       |     |
| Guadalajara |                                  |                     |                       |     |
| Toledo      |                                  |                     |                       |     |


Así, con esta tarjeta, sabiendo el histórico de consumo eléctrico de los hogares de Castilla-La Mancha, se podrá calcular el consumo para un año concreto. Además, con la producción anual eléctrica de energía que ya tiene en cuenta la meteorología de cada zona y la geografía, se podrá calcular cuántas placas solares fotovoltaicas se necesitarían instalar para abastecer el consumo eléctrico de los hogares.


--------------------------------------------

## Preparación del entorno

In [ ]:
import pandas as pd
import os 

SILVER_DATA_PATH = os.path.join("..", "data/silver/")
GOLD_DATA_PATH = os.path.join("..", "data/gold/")

In [ ]:
PROVINCIAS = ['Albacete', 'Ciudad Real', 'Cuenca', 'Guadalajara', 'Toledo']

## Datos

Primero, cargamos los datasets que vamos a necesitar para realizar el análisis:
* El dataset `consumo_electrico_clm.csv` que contiene el histórico del consumo eléctrico de Castilla-La Mancha desde el 2000 al 2023.
* El dataset de `produccion_lugar.csv` que contiene la producción teórica de un metro cuadrado de placa solar fotovoltaica con un rendimiento del 100%.

In [ ]:
consumo_df = pd.read_csv(os.path.join(SILVER_DATA_PATH, "consumo_electrico_clm.csv"), sep=";", encoding="latin1")
consumo_df.info()

In [ ]:
produccion_df = pd.read_csv(os.path.join(SILVER_DATA_PATH, "produccion_lugar.csv"), sep=";", encoding="latin1")
produccion_df.info()

## Procesamiento

### consumo_df

Con los datos de `consumo_df` queremos predecir el consumo eléctrico de Castilla-La Mancha en el año 2028. Para ello, vamos a realizar una regresión con los datos de consumo eléctrico de los años 2000 al 2023.

Los datos de `consumo_df` son los siguientes:

| Columna | Descripción | Unidades | Tipo de dato |
| --- | --- | --- | ---- |
| Año | Año de la observación |  | int64 |
| Mes | Nombre del mes de la observación |  | object |
| Albacete | Consumo eléctrico de la provincia de Albacete | MWh | float64 |
| Ciudad Real | Consumo eléctrico de la provincia de Ciudad Real | MWh | float64 |
| Cuenca | Consumo eléctrico de la provincia de Cuenca | MWh | float64 |
| Guadalajara | Consumo eléctrico de la provincia de Guadalajara | MWh | float64 |
| Toledo | Consumo eléctrico de la provincia de Toledo | MWh | float64 |

In [ ]:
consumo_df.drop(columns=["Unnamed: 0"], inplace=True)
consumo_df.head()

Se transforma la fecha a un formato datetime para establecerla como índice y poder representarla en un gráfico.

In [ ]:
encode_mes = {}
for i, mes in enumerate(consumo_df["Mes"].unique()):
    encode_mes[mes] = i + 1

# Reemplazar los nombres de los meses por números
consumo_df["Mes"] = consumo_df["Mes"].replace(encode_mes)

In [ ]:
# Combinar Año y Mes en una columna Fecha
consumo_df.rename(columns={'Año': 'year', 'Mes': 'month'}, inplace=True)
consumo_df["Fecha"] = pd.to_datetime(consumo_df[['month', 'year']].assign(day=1), yearfirst=False)

consumo_df.drop(columns=["year", "month"], inplace=True)
# Establecer la fecha como índice
consumo_df.set_index("Fecha", inplace=True)

Se realizará un análisis exploratorio de los datos mediante un gráfico de líneas.
* Albacete con el color azul
* Ciudad Real con el color naranja
* Cuenca con el color verde
* Guadalajara con el color rojo
* Toledo con el color morado


In [ ]:
from tkinter import font
import matplotlib.pyplot as plt

# Vector de colores para las provincias
colors = plt.cm.tab10(range(len(PROVINCIAS)))

# Crear un gráfico de lineas para mostrar fecha vs consumo para cada provincia
for provincia, color in zip(PROVINCIAS, colors):
    plt.figure(figsize=(20, 8))
    plt.plot(consumo_df.index, consumo_df[provincia], color=color)
    plt.title(f"Evolución del Consumo en {provincia}", fontsize=20)
    plt.xlabel("Fecha")
    plt.ylabel("Consumo (MWh)")
    plt.grid(alpha = 0.8)
    plt.tight_layout()
    plt.show()


In [ ]:
consumo_df.describe()

A continuación se muestra la gráfica de consumo para un año en una provincia, en este caso Albacete en el año 2000.

In [ ]:
albacete_2000 = consumo_df[:12]["Albacete"]
plt.figure(figsize=(14, 8))
plt.bar(albacete_2000.index, albacete_2000, width=20)
plt.title("Consumo de Albacete en el año 2000")
plt.xlabel("Mes")
plt.ylabel("Consumo (MWh)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Con este análisis exploratorio inicial, se observa que los datos presentan una gran variación y además hay valores atípicos, como en Ciudad Real y Guadalajara poco antes del año 2020. 

La gran variación de los datos se debe a su granularidad a nivel mensual. En los meses más fríos el consumo es mayor que en los meses más cálidos y ello, sumado a la variación entre años, hace difícil establecer una tendencia clara. Por ello, vamos a agrupar los datos por año para obtener una mejor predicción.

In [ ]:
consumo_anual = consumo_df.index.year
consumo_anual = consumo_df.groupby(consumo_anual).sum()
consumo_anual.head(2)

A continuación, se muestra de nuevo la gráfica del consumo por años.

In [ ]:
import matplotlib.pyplot as plt

# Colores 
colors = plt.cm.tab10(range(len(PROVINCIAS)))

# Crear un gráfico con todas las provincias juntas
plt.figure(figsize=(16, 10))
for provincia, color in zip(PROVINCIAS, colors):
    plt.plot(consumo_anual.index, consumo_anual[provincia], color=color, label=provincia)
    plt.xticks(consumo_anual.index)
    
plt.title(f"Evolución del Consumo en Castilla - La Mancha")
plt.xlabel("Año")
plt.ylabel("Consumo")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


Los datos muestran una tendencia creciente desde el año 2000 hasta el año 2009 aproximaadamente, a partir de entonces disminuye hasta el año 2014, en el que la mayoría de las provincias se mantienen o aumentan. Por esta razón, y para mejorar la precisión de la predicción, se seleccionarán solo los datos del 2014 a 2023.

In [ ]:
consumo_anual = consumo_anual[consumo_anual.index > 2013]

Se rota la tabla para que las provincias sean los índices y los años las columnas.

In [ ]:
consumo_anual = consumo_anual.transpose()
consumo_anual.head()

### produccion_df

Con los datos de `produccion_df` queremos calcular la producción de energía eléctrica para un panel solar de un metro cuadrado en un año. Para ello, vamos a filtrar los datos de las provincias de Castilla-La Mancha y agrupar los datos por año.

| Columna   | Descripción                                                            | Unidades | Tipo de dato |
| --------- | ---------------------------------------------------------------------- | -------- | ------------ |
| Provincia | Nombre de la provincia                                                 |          | object       |
| Potencia  | Producción de energía eléctrica de un panel solar de un metro cuadrado | MWh      | float64      |
| Mes       | Nombre del mes de la observación                                       |          | object       |


In [ ]:
# Filtrado por provincias de Castilla - La Mancha
produccion_clm = produccion_df[produccion_df["Provincia"].isin(PROVINCIAS)]
produccion_clm.head(3)

In [ ]:
print("Provincias: ", produccion_clm["Provincia"].unique())

Calculamos el total de la producción de un año para cada provincia, sumando la producción de cada mes.

In [ ]:
produccion_anual = produccion_clm.groupby("Provincia").sum()
produccion_anual.drop(columns=["Mes"], inplace=True)
produccion_anual

Este dato sería la producción ideal de energía, es decir, si la placa solar tuviera un rendimiento del 100%. Para obtener la producción real, tendríamos que multiplicar la producción ideal por el rendimiento de la placa solar que es del 80%, o lo que es lo mismo, tiene una eficiencia del 20%.

In [ ]:
produccion_anual = produccion_anual.apply(lambda x: x * 0.8, axis=1)
produccion_anual

In [ ]:
produccion_anual = produccion_anual.apply(lambda x: x / 1000, axis=1)
produccion_anual.rename({"Potencia kWh": "Potencia MWh"}, axis=1, inplace=True)
produccion_anual

### Resultado

Unimos los datos en un solo dataframe para obtener la tarjeta.

In [ ]:
tarjeta_5 = pd.concat([produccion_anual, consumo_anual], axis=1)
tarjeta_5.head()

Guardamos los datos en el archivo `gold/tarjeta_5.csv`.

In [ ]:
with open(GOLD_DATA_PATH + "tarjeta_5.csv", "w") as f:
    tarjeta_5.to_csv(f, sep=';', encoding='utf-8')

### Profiling de la tarjeta resultante

In [ ]:
from  ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(tarjeta_5, title="Tarjeta 5")
profile.to_file("tarjeta_5.html")